In [1]:
def parse(raw):
    ptr = 0
    data = {}
    depth = -1
    idx = -1
    need_new = True
    while ptr < len(raw):
        if raw[ptr]  == '[':
            depth += 1
            need_new = True
        elif raw[ptr] == ']':
            depth -= 1
            need_new = True   
        elif raw[ptr].isnumeric():
            num = int(raw[ptr])
            if need_new:
                idx += 1
                data[idx] = {} 
                data[idx]['depth'] = depth
                need_new = False
            if raw[ptr-1] == ',':  #have found the ',' in [[],b]
                data[idx]['b'] = num
            else:                  #have found a in [a,[[]]]
                data[idx]['a'] = num
        ptr += 1
    return data

def explode(data, k_explode):  #explode the kth item in data
    shrink = True
    if k_explode == 0:  #trival case #1 - far left
        if 'a' in data[k_explode+1].keys():
            data[k_explode+1]['a'] = data[k_explode+1]['a'] + data[k_explode]['b']
            data[k_explode]['depth'] -= 1
            data[k_explode]['a'] = 0
            del data[k_explode]['b']  
            shrink = False
        else:
            data[k_explode+1]['b'] = data[k_explode+1]['b'] + data[k_explode]['b']
            data[k_explode+1]['a'] = 0
    elif k_explode == (len(data.keys()) - 1): #trival case #2 - far right         
        if 'b' in data[k_explode-1].keys():
            data[k_explode-1]['b'] = data[k_explode-1]['b'] + data[k_explode]['a'] 
            data[k_explode]['depth'] -= 1
            data[k_explode]['b'] = 0
            del data[k_explode]['a']  
            shrink = False            
        else:
            data[k_explode-1]['a'] = data[k_explode-1]['a'] + data[k_explode]['a']
            data[k_explode-1]['b'] = 0
    else:
        if 'b' in data[k_explode-1].keys():      # [a,b],[[c,d],e] want b = b+c
            data[k_explode-1]['b'] = data[k_explode-1]['b'] + data[k_explode]['a']
        else:                            # [a,[[b,c],d] want a = a+b
            data[k_explode-1]['a'] = data[k_explode-1]['a'] + data[k_explode]['a']        
            
        if 'a' in data[k_explode+1].keys():   #[[a,[b,c]],[d,e]]   want d= d+c
            data[k_explode+1]['a'] = data[k_explode+1]['a'] + data[k_explode]['b']
        else:
            data[k_explode+1]['b'] = data[k_explode+1]['b'] + data[k_explode]['b']  #[[a,b],c] want c = c+b

        left = data[k_explode-1]['depth']
        current = data[k_explode]['depth']
        right = data[k_explode+1]['depth']

        if current - left == 1 and 'b' not in data[k_explode-1].keys():  # [[[],[a,[b,c]]],[]] --> [[[],[a+b,0]],[]]
            #assert 'b' not in data[k_explode-1].keys(), "Should not have a 'b' key"
            data[k_explode-1]['b'] = 0
        elif current - right == 1 and 'a' not in data[k_explode+1].keys(): # [[[],[[a,b],c]],[]] --> [[[],[0,b+c]],[]]
            #assert 'a' not in data[k_explode+1].keys(), "Should not have an 'a' key"
            data[k_explode+1]['a'] = 0 
        elif current == left and current != right:
            data[k_explode]['depth'] -= 1
            data[k_explode]['b'] = 0
            del data[k_explode]['a']
            return data            
        elif current == right and current != left:
            data[k_explode]['depth'] -= 1
            data[k_explode]['a'] = 0
            del data[k_explode]['b']
            return data
        
        else:                               
            assert 3==2, 'Broke it'
    if shrink:
        keys = list(data.keys())
        subtract = 0
        temp = {}
        for key in keys:
            if key == k_explode:
                subtract = 1
            else:
                temp[key-subtract] = data[key]
        data = temp    
    return data


def split(data, k_split):  #split the kth item in data
    keys = list(data[k_split].keys())
    a,b = -1,-1
    if 'a' in keys:
        a = data[k_split]['a']
    if 'b' in keys:
        b = data[k_split]['b']
    
    if a != -1 and a > 9:
        num = a
    else:
        num = b
      
    temp = {}
    temp['depth'] = data[k_split]['depth'] + 1
    temp['a'] = int( num / 2)
    temp['b'] = int((num / 2) + 0.5)        
    
    if a == num and b == -1:  # [a,[c,d]] --> [[a,b],[c,d]] = temp goes in place 
        data[k_split] = temp
    elif a == -1 and b == num:  #[[a,b],c] --> [[a,b],[c,d]] = temp goes in place
        data[k_split] = temp
    elif a == num:                                  #[a,b] --> [[a,b],c]
        del data[k_split]['a']
        add = 0
        new = {}        
        for key in range(len(data.keys())):
            if key == k_split:
                new[key+add] = temp
                add = 1
            new[key+add] = data[key]
        data = new       
    else:                                         #[a,b] --> [a,[b,c]]
        del data[k_split]['b']
        add = 0
        new = {}        
        for key in range(len(data.keys())):
            new[key+add] = data[key]
            if key == k_split:
                add = 1
                new[key+add] = temp 
        data = new
    return data

In [2]:
def reduce(raw, verbose = False):
    changed = True
    while changed == True:
        changed = False
        #first find if there is something at more than 4.  then explode first occurance of that depth
        max_depth = 0 
        for k,v in raw.items():
            d = v['depth']
            if d >= 4 and d > max_depth:
                max_depth = d

        if max_depth != 0:
            for k,v in raw.items():
                d = v['depth']
                if d == max_depth:
                    temp = raw
                    temp = explode(temp,k)
                    break
            raw = temp
            changed = True
            if verbose:
                print(f'\n\nExploded {k}')
                for k,v in raw.items():
                    print(k,v)

        if changed == False:      #didn't explode, now check for splits
            for k,v in raw.items():
                if ('a' in v.keys() and v['a'] > 9) or ('b' in v.keys() and v['b'] > 9):
                    temp = raw
                    temp = split(temp,k) 
                    changed = True
                    break
            if changed:
                raw = temp            
                if verbose:
                    print(f'\n\nSplit {k}') 
                    for k,v in raw.items():
                        print(k,v)   
    return raw

def add(d1, d2):
    data = {}
    biggest_key = max(list(d1.keys()))+1
    for k,v in d1.items():
        data[k] = v
        data[k]['depth'] += 1
    for k,v in d2.items():
        data[biggest_key+k] = v
        data[biggest_key+k]['depth'] += 1
    return data

  
def get_mag(data):
    changed = True
    
    while changed:
        changed = False
        max_depth = 0
        for k,v in data.items():
            #print(k,v)
            if v['depth'] > max_depth:
                max_depth = v['depth']
        #print('')
        for k,v in data.items():
            if v['depth'] == max_depth:
                c = 3*v['a'] + 2*v['b']
                changed = True
                break
        if changed:
            
            if k > 0:
                left = data[k-1]['depth']
            else:
                left = 999
            current = data[k]['depth']
            if k < len(data.keys())-1:
                right = data[k+1]['depth']
            else:
                right = 999

            shrink = False
            if current - left == 1 and 'b' not in data[k-1].keys():  # [[[],[a,[b,c]]],[]] --> [[[],[a+b,0]],[]]
                data[k-1]['b'] = c
                shrink = True
            elif current - right == 1 and 'a' not in data[k+1].keys(): # [[[],[[a,b],c]],[]] --> [[[],[0,b+c]],[]]
                data[k+1]['a'] = c
                shrink = True                    
            elif current == left and current != right:
                data[k]['depth'] -= 1
                data[k]['b'] = c
                del data[k]['a']     
            elif current == right and current != left:
                data[k]['depth'] -= 1
                data[k]['a'] = c
                del data[k]['b']       
            else:
                assert 3==2,'broken'

            if shrink:                
                temp = data
                for key,v in data.items():
                    if key > k:
                        temp[key-1] = temp[key]
                del temp[key]
                data = temp
            if len(data.keys())==1:
                data = 3*data[0]['a'] + 2*data[0]['b']
                changed = False
    return data

In [3]:
import numpy as np

inputs = []
with open('./input.txt','r') as file:
    for line in file:
        inputs.append(line.strip())
      
for idx, i in enumerate(inputs):
    if idx == 0:
        s1 = parse(i)
        s1 = reduce(s1)
    else:
        s2 = parse(i)
        s2 = reduce(s2)
        s1 = add(s1,s2)
        s1 = reduce(s1)

get_mag(s1)      



3486

In [4]:
max_number = 0
for ia, a in enumerate(inputs):
    for ib, b in enumerate(inputs):
        if ia == ib:
            continue
        s1 = parse(a)
        s1 = reduce(s1)
        s2 = parse(b)
        s2 = reduce(s2)
        s2 = add(s1,s2)
        s2 = reduce(s2)
        s2 = get_mag(s2)
        if s2>max_number:
            max_number = s2
max_number

4747